In [1]:
import numpy as np
import scipy as sp
from utility import get_best_connected, invert, score, get_highest_out_degrees 
#from data_loader import M,F, benchmark_mapping
rng = np.random.default_rng(0)

In [287]:
from utility import random_graph_dist,shuffle_graph,shuffle_map

N=500
def mod_pow(x,m,p):
   y=x.copy()
   for i in range(m-1):
      y = (y*x)%p

   return y

V=np.arange(N)
G=(mod_pow(V[:,None]-V[None,:],127,509)==1).astype(np.int32)
   
H,shuffle=shuffle_graph(G,stable=None,rng=rng)
mapping = shuffle_map(shuffle,stable=None)
inverse_mapping = invert(mapping)

In [288]:
np.count_nonzero(G)

62377

In [308]:
from algorithms import move_swap_optimization

penalty_gradient = lambda usage: 2048*usage
rng = np.random.default_rng(0)

mapping, score_track = move_swap_optimization(G,H,mapping,penalty_gradient,rng=rng,verbose=True)
inverse_mapping, inverse_score = move_swap_optimization(H,G,inverse_mapping,penalty_gradient,rng=rng,verbose=True)

score(G,H,mapping,require_surjective=False), score(H,G,inverse_mapping,require_surjective=False), score(G,G,np.arange(N))

(20872, 20770, 62377)

In [309]:
good_points = (inverse_mapping[mapping] == np.arange(N))
good_points = np.flatnonzero(good_points)
good_points, mapping[good_points],shuffle[good_points]

(array([], dtype=int64), array([], dtype=int32), array([], dtype=int32))

In [307]:
mapping=shuffle_map(mapping,good_points)
inverse_mapping=shuffle_map(inverse_mapping,mapping[good_points])

In [266]:
from algorithms import move_swap_optimization

k=20
N = G.shape[0]
trials = 200

G_mappings_list, H_mappings_list = [],[]

for g in range(trials):

    K = random_graph_dist(k,p=0.5,sample_source=lambda : rng.poisson(lam=1), rng=rng)

    start_mapping = rng.choice(k,size=N)
    projection,_ = move_swap_optimization(G,K,start_mapping=start_mapping,penalty_gradient=lambda x:x)
    G_mappings_list.append(projection)

    start_mapping = start_mapping[invert(mapping)]
    projection,_ = move_swap_optimization(H,K,start_mapping=start_mapping,penalty_gradient=lambda x:x)
    H_mappings_list.append(projection)

    print(f"{g=}",end="\r")
    


In [263]:
G_mappings = np.array(G_mappings_list).T
H_mappings = np.array(H_mappings_list).T
unshuffled_H_mappings = H_mappings[shuffle,:]
(G_mappings==H_mappings).sum(),(G_mappings==unshuffled_H_mappings).sum()

(1072, 1351)

In [264]:
table=(G_mappings[:,None,:]==unshuffled_H_mappings[None,:,:]).sum(axis=2)
table.argmax(axis=1)==shuffle

array([False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])